# Программирование для всех<br>(основы работы с Python)

*Тамбовцева А.А.*

## Практикум 6.1. Введение в парсинг HTML

### Постановка задачи

Есть реально существующая [страница](https://elementy.ru/problems/3181/Sekrety_tablitsy_Pifagora) со статьей по математике для школьников.

Исходный код данной страницы достаточно громоздкий из-за отсылок к стилевым файлам, фрагментам кода JavaScript для реализации разных процессов и дополнительной информации, которую в режиме пользователя мы не видим. Поэтому была написана упрощенная [версия](https://html-preview.github.io/?url=https://github.com/allatambov/PyAll25/blob/main/page0601.html) этой страницы. В этой версии убраны элементы, не относящиеся непосредственно к статье, плюс, без специальных настроек формулы не отображаются красиво. Это нормально, так должно и быть.

Чтобы увидеть исходный код упрощенной версии страницы для изучения и отправки запроса, используйте следующую ссылку: https://raw.githubusercontent.com/allatambov/PyAll25/refs/heads/main/page0601.html.

Для выполнения заданий необходимо ознакомиться с полным исходным кодом страницы, инструменты разработчика здесь не сработают, поскольку упрощенная версия страницы лежит на Github как обычный текст. Другими словами, нужно просто посмотреть, что в каких тэгах находится, сопоставляя [это](https://html-preview.github.io/?url=https://github.com/allatambov/PyAll25/blob/main/page0601.html) и [это](https://raw.githubusercontent.com/allatambov/PyAll25/refs/heads/main/page0601.html).

### Подготовка к работе

Импортируем необходимые библиотеки. Нам понадобится модуль `requests` для отправки запросов, для «подключения» к странице и получения её содержимого в виде строки, и функция `BeautifulSoup` из библиотеки `bs4` для удобного поиска по полученной строке:

In [1]:
import requests
from bs4 import BeautifulSoup

Подключаемся к странице сайта, ссылка на которую сохранена в переменной `link`:

In [2]:
link = "https://raw.githubusercontent.com/allatambov/PyAll25/refs/heads/main/page0601.html"

# verify = False для выключения проверки сертификатов
page = requests.get(link, verify = False)
print(page)

/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1107: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  InsecureRequestWarning,


<Response [200]>


Объект, который мы сохранили в `page`, имеет особый тип `requests.models.Response`, он же ответ на запрос:

In [3]:
print(type(page))

<class 'requests.models.Response'>


Сам по себе объект полностью мы увидеть не можем, но из него можно извлечь разные атрибуты – характеристики объекта. Например, статус ответа, который мы и так увидели выше:

In [4]:
print(page.status_code)

200


Код 200 означает, что запрос получен, обработан, и ответ на него получен. Другими словами, у нас нет проблем с доступом к странице (потенциальная проблема со стороны сервера, например, ведущиеся технические работы или отказ в правах доступа) и нет проблем с самим запросом (потенциальная проблема со стороны пользователя, например, «битая» ссылка). Про коды статусов можно почитать в [статье](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

Аналогичным образом можем узнать кодировку страницы:

In [5]:
print(page.encoding)

utf-8


И, самое главное, весь исходный код HTML в виде большой строки с текстом:

In [6]:
print(type(page.text))
print(page.text)

<class 'str'>
<!DOCTYPE html>
<html>
	<head>
		<title>Секреты таблицы Пифагора • Николай Авилов • Научно-популярные задачи на «Элементах» • Математика</title>
		<META name="description" content="На задней обложке школьных тетрадей часто печатают таблицу 
		Пифагора — квадратную таблицу 9×9, в которой строки и столбцы пронумерованы числами от 1 до 9, 
		а на пересечениях строк и столбцов стоят произведения соответствующих чисел. 
		Найдите, чему равна сумма всех чисел таблицы умножения?"/>
		<link rel="icon" type="image/png" href="https://elementy.ru/images/eltnewdesign/logo16bw.png"  sizes="16x16"/>
	</head>
	<body>
		<h1>Секреты таблицы Пифагора</h1>
		<div class="sublink">
			<span class="date">13.01.2025</span>
			<span class="break">•</span> 
			<a href="/problems/a/5276093/Nikolay_Avilov">Николай Авилов</a>
			<span class="break">•</span>
			<a href="/problems/t/21097/Matematika">Математика</a>
		</div>

		<h2>Задача</h2>
		<div  style='width:250px'>
			<div align=center class='im

Выполнять поиск по такой строке с тэгами не очень удобно (даже если вы знакомы с регулярными выражениями), поэтому преобразуем строку в объект типа `BeautifulSoup`. Такой объект внешне несильно отличается от обычной строки, однако внутри по структуре похож на словарь, и это сходство значительно упрощает поиск по тэгам и атрибутам.

In [8]:
soup = BeautifulSoup(page.text)
print(type(soup))
print(soup)

<class 'bs4.BeautifulSoup'>
<!DOCTYPE html>
<html>
<head>
<title>Секреты таблицы Пифагора • Николай Авилов • Научно-популярные задачи на «Элементах» • Математика</title>
<meta content="На задней обложке школьных тетрадей часто печатают таблицу 
		Пифагора — квадратную таблицу 9×9, в которой строки и столбцы пронумерованы числами от 1 до 9, 
		а на пересечениях строк и столбцов стоят произведения соответствующих чисел. 
		Найдите, чему равна сумма всех чисел таблицы умножения?" name="description"/>
<link href="https://elementy.ru/images/eltnewdesign/logo16bw.png" rel="icon" sizes="16x16" type="image/png"/>
</head>
<body>
<h1>Секреты таблицы Пифагора</h1>
<div class="sublink">
<span class="date">13.01.2025</span>
<span class="break">•</span>
<a href="/problems/a/5276093/Nikolay_Avilov">Николай Авилов</a>
<span class="break">•</span>
<a href="/problems/t/21097/Matematika">Математика</a>
</div>
<h2>Задача</h2>
<div style="width:250px">
<div align="center" class="img">
<img alt="Рис. 1." bo

### Примеры поиска по тэгам

На объектах `BeautifulSoup` определены методы `.find()` и `.find_all()`. Оба метода возвращают фрагменты кода HTML, которые соответствуют критериям поиска, различие заключается в том, что метод `.find()` предназначен для поиска одного совпадения (если критериям поиска соответствует несколько элементов на странице, то берётся только первый), а метод `.find_all()` – для поиска всех совпадений. В первом случае результат возвращается в виде одного элемента типа `BeautifulSoup`, а во втором – в виде списка таких элементов. Давайте попробуем что-то поискать!

Найдем заголовок статьи. Если внимательно посмотреть на код страницы, заметим, что он находится внутри тэга `<h1>`. Так как заголовок у статьи обычно один, задействуем метод `.find()`, который возвращает один результат:

In [9]:
soup.find("h1")

<h1>Секреты таблицы Пифагора</h1>

Результат – фрагмент кода HTML со всеми тэгами. Как получить «чистый» текст? Запросить текст с помощью атрибута `.text`:

In [10]:
title = soup.find("h1").text
print(title)

Секреты таблицы Пифагора


Найдем теперь подзаголовки статьи – здесь это *Задача*, *Подсказка*, *Решение* и *Послесловие*. Если вновь внимательно посмотрим на исходный код страницы, заметим, что они находятся в тэгах `<h2>`. Так как их несколько, задействуем метод `.find_all()`, который возвращает все результаты поиска списком:

In [11]:
soup.find_all("h2")

[<h2>Задача</h2>, <h2>Подсказка</h2>, <h2>Решение</h2>, <h2>Послесловие</h2>]

Каждый элемент списка – объект типа `BeautifulSoup` с тэгами, придется забрать атрибут `.text` из каждого элемента списка через списковое включение:

In [12]:
raw_headings = soup.find_all("h2")
headings = [h.text for h in raw_headings]
print(headings)

['Задача', 'Подсказка', 'Решение', 'Послесловие']


### Задача 1

Выполнив поиск по объекту `soup`, найдите и сохраните в список **все абзацы** с текстом статьи (без заголовков, подзаголовков, дополнительных подписей). Элементы этого списка должны быть обычными строками с «чистым» текстом без тэгов.

Объедините элементы списка в одну строку с полным текстом статьи, сохраните его в переменную `text`.

In [13]:
p_raw = soup.find_all("p")[1:]

# извлечем текст через .text из каждого элемента
# и выполним замену лишних символов для переносов и отступов

pars = [p.text.replace("\n", " ").replace("\t", " ") for p in p_raw]
text = " ".join(pars)
text

'На задней обложке школьных тетрадей часто печатают таблицу    Пифагора — квадратную таблицу 9×9, в которой строки и столбцы    пронумерованы числами от 1 до 9, а на пересечениях строк    и столбцов стоят произведения соответствующих чисел. Это    удобный и компактный способ записать результат умножения чисел,    с которыми школьникам чаще всего приходится иметь дело. У таблицы Пифагора есть несколько секретов, о них мы    и поговорим. Для начала найдите, чему равна сумма    всех чисел таблицы умножения? Пожалуй, это тот случай, когда подсказка толком    и не требуется. Есть простое объяснение, почему сейчас    предложена эта задача. Задача несложная и доступна, пожалуй, даже младшим    школьникам. Проще всего ее решить непосредственным сложением    всех чисел от 1 до 81. Но скучную, на первый взгляд, задачу суммирования    чисел можно решить,       получив при этом удовольствие. Например, суммированием чисел построчно.       В первой строке сумма чисел равна \\(1+2+3+\\ldots +9\\). По

### Задача 2

Выполнив поиск по объекту `soup`, любым способом найдите и сохраните в переменную `author` **имя и фамилию автора** статьи (без лишних пробелов и символов в строке).

In [14]:
author = soup.find_all("a")[0].text
print(author)

Николай Авилов


### Задача 3

Выполнив поиск по объекту `soup`, любым способом найдите и сохраните в переменную `date` **дату** статьи.

In [15]:
date = soup.find_all("div")[0].find("span").text
print(date)

13.01.2025


### Задача 4

Объедините информацию, полученную в предыдущих задачах, в словарь вида:

    {"title" : значение1,
     "heads" : значение2, 
     "text" : значение3, 
     "author" : значение4, 
     "date" : значение5},
     
где вместо `значение1`-`значение5` подставлены значения из переменных `title`, `headings` `text`, `author` и `date`, созданных ранее.

In [16]:
info = {"title" : title, 
       "heads" : headings, 
       "text" : text, 
       "author" : author, 
       "date" : date}
info

{'title': 'Секреты таблицы Пифагора',
 'heads': ['Задача', 'Подсказка', 'Решение', 'Послесловие'],
 'text': 'На задней обложке школьных тетрадей часто печатают таблицу    Пифагора — квадратную таблицу 9×9, в которой строки и столбцы    пронумерованы числами от 1 до 9, а на пересечениях строк    и столбцов стоят произведения соответствующих чисел. Это    удобный и компактный способ записать результат умножения чисел,    с которыми школьникам чаще всего приходится иметь дело. У таблицы Пифагора есть несколько секретов, о них мы    и поговорим. Для начала найдите, чему равна сумма    всех чисел таблицы умножения? Пожалуй, это тот случай, когда подсказка толком    и не требуется. Есть простое объяснение, почему сейчас    предложена эта задача. Задача несложная и доступна, пожалуй, даже младшим    школьникам. Проще всего ее решить непосредственным сложением    всех чисел от 1 до 81. Но скучную, на первый взгляд, задачу суммирования    чисел можно решить,       получив при этом удовольствие.

### Задача 5

Найдите в исходном тексте все фрагменты с тэгом `<mark>`. Извлеките текст и составьте слово – сохраните его в переменную :)

In [17]:
soup.find_all("mark")

[<mark style="background:white!important">мо</mark>,
 <mark style="background:white!important">л</mark>,
 <mark style="background:white!important">од</mark>,
 <mark style="background:white!important">цы</mark>]

In [18]:
letters = [m.text for m in soup.find_all("mark")] 
word = "".join(letters)
print(word)

молодцы
